# QuiverTools showcase

First, we install QuiverTools.jl in a clean environment.

In [41]:
using Pkg;
Pkg.activate("QuiverTools-showcase")
# Pkg.add(url="https://github.com/QuiverTools/QuiverTools.jl")
using QuiverTools

  Activating project at `~/Documents/notes/quivertools-showcase/QuiverTools-showcase`


In [71]:
Q = mKronecker_quiver(3)

3-Kronecker quiver, with adjacency matrix [0 3; 0 0]

In [72]:
println("Adjacency matrix: ", Q.adjacency)
println("arrows: ", arrows(Q))
println("Number of vertices: ", nvertices(Q))
println("Number of arrows: ", narrows(Q))

Adjacency matrix: [0 3; 0 0]
arrows: [[1, 2], [1, 2], [1, 2]]
Number of vertices: 2
Number of arrows: 3


### Basic functionalities of directed graphs are implemented

In [73]:
is_connected(Q)

true

In [74]:
is_acyclic(Q)

true

In [75]:
println("Matrix of the Euler form: ",QuiverTools.Euler_matrix(Q))
Euler_form(Q, [2, 3], [2, 3])

Matrix of the Euler form: [1 -3; 0 1]


-5

### We construct a QuiverModuli object

In [76]:
M = QuiverModuliSpace(Q, [2, 3])

Moduli space of semistable representations of 3-Kronecker quiver, with adjacency matrix [0 3; 0 0]
      with dimension vector [2, 3] and stability parameter [9, -6]

### We can compute the Hodge polynomial, as well as the Poincaré polynomial of a given quiver moduli:

In [77]:
H = Hodge_polynomial(M)

x^6*y^6 + x^5*y^5 + 3*x^4*y^4 + 3*x^3*y^3 + 3*x^2*y^2 + x*y + 1

In [78]:
dimension(M)

6

In [79]:
P = Poincare_polynomial(M)

L^6 + L^5 + 3*L^4 + 3*L^3 + 3*L^2 + L + 1

In [80]:
println("Picard rank: ",Picard_rank(M))
println("Index: ",index(M))

Picard rank: 1
Index: 3


### We can study the unstable locus of the parameter space:

In [89]:
Q = mKronecker_quiver(3); M = QuiverModuliSpace(Q, [2, 3], [3, -2]);
all_HN_types(M)

8-element Vector{Vector{AbstractVector{Int64}}}:
 [[2, 3]]
 [[1, 1], [1, 2]]
 [[2, 2], [0, 1]]
 [[2, 1], [0, 2]]
 [[1, 0], [1, 3]]
 [[1, 0], [1, 2], [0, 1]]
 [[1, 0], [1, 1], [0, 2]]
 [[2, 0], [0, 3]]

In [87]:
# slightly improves performance
all_HN_types(M, unstable=true, ordered=false)

7-element Vector{Vector{AbstractVector{Int64}}}:
 [[1, 0], [1, 3]]
 [[1, 0], [1, 1], [0, 2]]
 [[1, 0], [1, 2], [0, 1]]
 [[2, 0], [0, 3]]
 [[1, 1], [1, 2]]
 [[2, 1], [0, 2]]
 [[2, 2], [0, 1]]

In [88]:
all_Teleman_bounds(M)

Dict{Vector{AbstractVector{Int64}}, Int64} with 7 entries:
  [[2, 2], [0, 1]]         => 60
  [[2, 1], [0, 2]]         => 100
  [[1, 0], [1, 2], [0, 1]] => 100
  [[1, 0], [1, 3]]         => 360
  [[1, 0], [1, 1], [0, 2]] => 270
  [[1, 1], [1, 2]]         => 15
  [[2, 0], [0, 3]]         => 270

In [39]:
all_weights_endomorphisms_universal_bundle(M)

Dict{Vector{AbstractVector{Int64}}, Vector{Int64}} with 7 entries:
  [[2, 2], [0, 1]]         => [0, 15, -15, 0]
  [[2, 1], [0, 2]]         => [0, 10, -10, 0]
  [[1, 0], [1, 2], [0, 1]] => [0, 10, 15, -10, 0, 5, -15, -5, 0]
  [[1, 0], [1, 3]]         => [0, 45, -45, 0]
  [[1, 0], [1, 1], [0, 2]] => [0, 15, 30, -15, 0, 15, -30, -15, 0]
  [[1, 1], [1, 2]]         => [0, 5, -5, 0]
  [[2, 0], [0, 3]]         => [0, 15, -15, 0]